### Selected Word Group In Youtube Sentence

In [29]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [30]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected Remain 204"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_Remain_204"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [31]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [32]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [33]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [34]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [35]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [36]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [37]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [38]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [39]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [40]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,3831.058,3833.058,ben uçak gönderdim,LmY6NZwmSa4
1,3718.780,3720.080,reis arıyor,Lm12f30wJ1I
2,3771.680,3774.870,senin ekip başkaları artık abi değil mi asım,J3lK1h2j7Lo
3,757.320,759.380,ezer yakar geçerim,SmIaXMtgO2I
4,556.420,559.180,tamam bunu deneyeceğim onu deneyeceğim okay be...,TOo7_nW4dfI
...,...,...,...,...
3934202,58.205,60.205,come here please,jiJ6sIVUBvI
3934203,815.138,819.543,asıl sen onu hiç tanımamışsın bir daha annemi ...,ioHd1Rcp5Ew
3934204,31.008,36.485,ülkedeki ekonomi neredeyse berbat olduğundan h...,EGvPvaxYTyU
3934205,12345.391,12350.740,devletler hakkındaki allahın sünneti böyledir ...,grKYOG3pTpI


In [41]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,adam yine geldi
1,al bakalım dostum
2,al şunu al
3,al şunu da
4,ama asla olmaz
...,...
199,yok tamam mı
200,yoksa bizi de
201,yoksa bugün mü
202,zaman bugün mü


In [ ]:
# Option 
# Skip if it doesn't need
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [42]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [43]:
len(search_list)

204

In [44]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,senin ile ilgili,2341.013,2343.655,senin ile ilgili yazıyı göndermişlerdi bu sabah,pdtUajPsAa0
1,senin ile ilgili,4334.120,4337.000,senin ile ilgili her şey ashabımı bozuyor,sGPdUbK7nKQ
2,senin ile ilgili,3034.625,3037.045,elimde senin ile ilgili bir sürü delil var,jKXIyTewiMs
3,senin ile ilgili,1166.070,1168.019,ben senin ile ilgili hiç bir şey bilmiyorum,C2xtDX5qexc
4,senin ile ilgili,1798.920,1801.580,güzel plan benimde senin ile ilgili bir planım...,xzD8szESJP8
...,...,...,...,...,...
644,seninle birlikte mi,4275.620,4277.700,biz seninle birlikte mi yaşayacağız,TW9Ij2Lr3U4
645,seninle birlikte mi,2477.029,2479.415,ben seninle birlikte mi yaşayacağım yani,VJFduvcKW1M
646,seninle birlikte mi,1324.640,1327.940,selena hanım kızım şimdi biz bu evde seninle b...,aMGS7vyx4Zo
647,bir küçük kız,271.373,274.003,ama bu adamın sahiplenmediği bir küçük kız çoc...,WL496X8YpS0


In [45]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,senin ile ilgili,2341.013000,2341.968617,senin ile ilgili yazıyı göndermişlerdi bu sabah,pdtUajPsAa0
1,senin ile ilgili,4334.120000,4335.314146,senin ile ilgili her şey ashabımı bozuyor,sGPdUbK7nKQ
2,senin ile ilgili,3034.970714,3036.007857,elimde senin ile ilgili bir sürü delil var,jKXIyTewiMs
3,senin ile ilgili,1166.205977,1167.021837,ben senin ile ilgili hiç bir şey bilmiyorum,C2xtDX5qexc
4,senin ile ilgili,1799.877600,1800.835200,güzel plan benimde senin ile ilgili bir planım...,xzD8szESJP8
...,...,...,...,...,...
644,seninle birlikte mi,4275.798286,4277.046286,biz seninle birlikte mi yaşayacağız,TW9Ij2Lr3U4
645,seninle birlikte mi,2477.207950,2478.460600,ben seninle birlikte mi yaşayacağım yani,VJFduvcKW1M
646,seninle birlikte mi,1326.413134,1327.447463,selena hanım kızım şimdi biz bu evde seninle b...,aMGS7vyx4Zo
647,bir küçük kız,272.762434,273.506774,ama bu adamın sahiplenmediği bir küçük kız çoc...,WL496X8YpS0


In [46]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,senin ile ilgili,2340.413000,2342.568617,senin ile ilgili yazıyı göndermişlerdi bu sabah,pdtUajPsAa0
1,senin ile ilgili,4333.520000,4335.914146,senin ile ilgili her şey ashabımı bozuyor,sGPdUbK7nKQ
2,senin ile ilgili,3034.370714,3036.607857,elimde senin ile ilgili bir sürü delil var,jKXIyTewiMs
3,senin ile ilgili,1165.605977,1167.621837,ben senin ile ilgili hiç bir şey bilmiyorum,C2xtDX5qexc
4,senin ile ilgili,1799.277600,1801.435200,güzel plan benimde senin ile ilgili bir planım...,xzD8szESJP8
...,...,...,...,...,...
644,seninle birlikte mi,4275.198286,4277.646286,biz seninle birlikte mi yaşayacağız,TW9Ij2Lr3U4
645,seninle birlikte mi,2476.607950,2479.060600,ben seninle birlikte mi yaşayacağım yani,VJFduvcKW1M
646,seninle birlikte mi,1325.813134,1328.047463,selena hanım kızım şimdi biz bu evde seninle b...,aMGS7vyx4Zo
647,bir küçük kız,272.162434,274.106774,ama bu adamın sahiplenmediği bir küçük kız çoc...,WL496X8YpS0


In [47]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,senin ile ilgili,2340,2343,senin ile ilgili yazıyı göndermişlerdi bu sabah,pdtUajPsAa0
1,senin ile ilgili,4334,4336,senin ile ilgili her şey ashabımı bozuyor,sGPdUbK7nKQ
2,senin ile ilgili,3034,3037,elimde senin ile ilgili bir sürü delil var,jKXIyTewiMs
3,senin ile ilgili,1166,1168,ben senin ile ilgili hiç bir şey bilmiyorum,C2xtDX5qexc
4,senin ile ilgili,1799,1801,güzel plan benimde senin ile ilgili bir planım...,xzD8szESJP8
...,...,...,...,...,...
644,seninle birlikte mi,4275,4278,biz seninle birlikte mi yaşayacağız,TW9Ij2Lr3U4
645,seninle birlikte mi,2477,2479,ben seninle birlikte mi yaşayacağım yani,VJFduvcKW1M
646,seninle birlikte mi,1326,1328,selena hanım kızım şimdi biz bu evde seninle b...,aMGS7vyx4Zo
647,bir küçük kız,272,274,ama bu adamın sahiplenmediği bir küçük kız çoc...,WL496X8YpS0


In [48]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,senin ile ilgili,2340,2343,senin ile ilgili yazıyı göndermişlerdi bu sabah,pdtUajPsAa0,https://www.youtube.com/watch?v=pdtUajPsAa0&t=...
1,senin ile ilgili,4334,4336,senin ile ilgili her şey ashabımı bozuyor,sGPdUbK7nKQ,https://www.youtube.com/watch?v=sGPdUbK7nKQ&t=...
2,senin ile ilgili,3034,3037,elimde senin ile ilgili bir sürü delil var,jKXIyTewiMs,https://www.youtube.com/watch?v=jKXIyTewiMs&t=...
3,senin ile ilgili,1166,1168,ben senin ile ilgili hiç bir şey bilmiyorum,C2xtDX5qexc,https://www.youtube.com/watch?v=C2xtDX5qexc&t=...
4,senin ile ilgili,1799,1801,güzel plan benimde senin ile ilgili bir planım...,xzD8szESJP8,https://www.youtube.com/watch?v=xzD8szESJP8&t=...
...,...,...,...,...,...,...
644,seninle birlikte mi,4275,4278,biz seninle birlikte mi yaşayacağız,TW9Ij2Lr3U4,https://www.youtube.com/watch?v=TW9Ij2Lr3U4&t=...
645,seninle birlikte mi,2477,2479,ben seninle birlikte mi yaşayacağım yani,VJFduvcKW1M,https://www.youtube.com/watch?v=VJFduvcKW1M&t=...
646,seninle birlikte mi,1326,1328,selena hanım kızım şimdi biz bu evde seninle b...,aMGS7vyx4Zo,https://www.youtube.com/watch?v=aMGS7vyx4Zo&t=...
647,bir küçük kız,272,274,ama bu adamın sahiplenmediği bir küçük kız çoc...,WL496X8YpS0,https://www.youtube.com/watch?v=WL496X8YpS0&t=...


In [49]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

127

In [50]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [51]:
search_all_set.difference(search_result_set)

{'adam yine geldi',
 'al bakalım dostum',
 'ama asla olmaz',
 'ama biliyorum işte',
 'ama zaten biliyorsun',
 'artık hiçbir zaman',
 'aslında gerçek şu',
 'bak gerçekten üzgünüm',
 'bana ver şunu',
 'ben bile değilim',
 'biliyorum işte tamam',
 'biri olabilir diye',
 'birlikte olacak mısın',
 'bu asla olmaz',
 'bu fazla uzun',
 'bu harika değil',
 'bu harika olacak',
 'bu küçük adam',
 'bu sorun değil',
 'bu yeni geldi',
 'bugün mü oldu',
 'buna izin var',
 'bunu biliyorum işte',
 'bunu nasıl biliyorsun',
 'bunu tekrar söyle',
 'bunu zaten biliyorsun',
 'burada bile değilim',
 'daha asla olmaz',
 'evet efendim ama',
 'evet efendim biliyorum',
 'evet harika olur',
 'evet sadece biraz',
 'fazla uzun değil',
 'gece geliyor musun',
 'gerçek bile değilim',
 'gerçekten harika biri',
 'gerçekten harika olur',
 'gerçekten üzgünüm ama',
 'hala orada olabilir',
 'hayır onlar bizim',
 'hayır siz devam',
 'hemen geri istiyorum',
 'hey hadi ama',
 'iki kız vardı',
 'iyi iş yapıyorsun',
 'işte yine 

In [52]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,ver,10
2,burada,9
1,şunu,9
3,hayır,7
4,o,7
...,...,...
119,şeyi,1
120,geldi,1
121,eğer,1
122,tabii,1


In [53]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [54]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [55]:
set_word.difference(set_count)

{'artık',
 'baba',
 'belki',
 'beni',
 'biz',
 'bizi',
 'buna',
 'büyük',
 'de',
 'demek',
 'dostum',
 'ederim',
 'en',
 'gerçekten',
 'git',
 'hakkında',
 'hala',
 'her',
 'hiç',
 'hiçbir',
 'ilk',
 'iş',
 'kadar',
 'kendi',
 'ki',
 'kimse',
 'merhaba',
 'oh',
 'ol',
 'olan',
 'olarak',
 'olsun',
 'onları',
 'para',
 'pekala',
 'sanırım',
 'selam',
 'seni',
 'sizi',
 'son',
 'sonra',
 'tek',
 'tekrar',
 'teşekkür',
 'teşekkürler',
 'tüm',
 'uzun',
 'var',
 'yeni',
 'yüzden',
 'önce',
 'önemli',
 'üzgünüm',
 'şekilde'}

In [56]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [27]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_67_Threegram_Remain_Word_10_Youtube_0.6s_Timeshift_Result2.xlsx',
 'Turkish_10_Threegram_Remain_Word2_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 